In [1]:
import boto3

import os
from dotenv import load_dotenv

load_dotenv()

session = boto3.Session(
    aws_access_key_id=os.getenv('BEDROCK_AWS_ACCESS_KEY'),
    aws_secret_access_key=os.getenv('BEDROCK_AWS_SECRET_KEY'),
)

client = session.client("bedrock-runtime", "us-west-2")

In [3]:
from langchain_aws.chat_models import BedrockChat

chat = BedrockChat(
    client=client,
    model_id="meta.llama3-1-405b-instruct-v1:0",
    model_kwargs={
        "temperature":0.1
    }
)

chat.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'usage': {'prompt_tokens': 12, 'completion_tokens': 10, 'total_tokens': 22}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-1-405b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 12, 'completion_tokens': 10, 'total_tokens': 22}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-1-405b-instruct-v1:0'}, id='run-3850bf13-fe9d-4bc9-b628-f29467bd06c7-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

In [5]:
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

full_template = """
{introduction}

{example}

{start}
"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """
Here's an example of an interaction:
Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """
Now, do this for real!
Q: {input}
A:
"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

pipeline_prompt.input_variables

['person', 'example_a', 'example_q', 'input']

In [6]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)
print("==================== AI SAYING ====================")
chat.invoke(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)


You are impersonating Elon Musk.


Here's an example of an interaction:
Q: What's your favorite car?
A: Tesla


Now, do this for real!
Q: What's your favorite social media site?
A:


==================== AI SAYING ====================


AIMessage(content="Twitter, of course! It's the best way to connect directly with the people and share my thoughts on the future of humanity. And, let's be real, it's also a great way to troll the haters and short sellers.", additional_kwargs={'usage': {'prompt_tokens': 58, 'completion_tokens': 49, 'total_tokens': 107}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-1-405b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 58, 'completion_tokens': 49, 'total_tokens': 107}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-1-405b-instruct-v1:0'}, id='run-4d3aaccf-3e3e-41c2-9186-02b0e4d78868-0', usage_metadata={'input_tokens': 58, 'output_tokens': 49, 'total_tokens': 107})